In [43]:
import numpy as np
import pandas as pd
import requests
import json

from sklearn.linear_model import LinearRegression

In [25]:
def add_row(df, category, ticker, value, ipca, cdi, ref):
    df.loc[len(df)] = {'Ticker': ticker, 'Categoria': category, 'Valor': value, 
                       'IPCA': ipca, 'CDI': cdi, 'Ref': ref}
    
    return df

In [26]:
df = pd.DataFrame(columns=['Ticker', 'Categoria', 'Valor', 'IPCA', 'CDI', 'Ref'])

df = add_row(df, 'Kinea', 'KNIP11', 92, 8.24, np.nan, '24/07/23')
df = add_row(df, 'Kinea', 'KNIP11', 93, 8.12, np.nan, '24/07/23')

df = add_row(df, 'Kinea', 'KNSC11', 86, 8.17, np.nan, '24/07/23')
df = add_row(df, 'Kinea', 'KNSC11', 87, 7.91, np.nan, '24/07/23')

df = add_row(df, 'Kinea', 'KNHY11', 98, 10.40, np.nan, '24/07/23')
df = add_row(df, 'Kinea', 'KNHY11', 99, 10.12, np.nan, '24/07/23')

df = add_row(df, 'Kinea', 'KCRE11', 95, 8.44, np.nan, '24/07/23')
df = add_row(df, 'Kinea', 'KCRE11', 96, 8.28, np.nan, '24/07/23')

df = add_row(df, 'Kinea', 'KNCR11', 96, np.nan, 2.83, '24/07/23')
df = add_row(df, 'Kinea', 'KNCR11', 97, np.nan, 2.57, '24/07/23')

df = add_row(df, 'Kinea', 'KNCA11', 105, np.nan, 2.76, '24/07/23')
df = add_row(df, 'Kinea', 'KNCA11', 106, np.nan, 2.43, '24/07/23')

In [27]:
df

,Ticker,Categoria,Valor,IPCA,CDI,Ref
0,KNIP11,Kinea,92,8.24,NaN,24/07/23
1,KNIP11,Kinea,93,8.12,NaN,24/07/23
2,KNSC11,Kinea,86,8.17,NaN,24/07/23
3,KNSC11,Kinea,87,7.91,NaN,24/07/23
4,KNHY11,Kinea,98,10.40,NaN,24/07/23
5,KNHY11,Kinea,99,10.12,NaN,24/07/23
6,KCRE11,Kinea,95,8.44,NaN,24/07/23
7,KCRE11,Kinea,96,8.28,NaN,24/07/23
8,KNCR11,Kinea,96,NaN,2.83,24/07/23
9,KNCR11,Kinea,97,NaN,2.57,24/07/23


In [31]:
regr = {ticker: LinearRegression() for ticker in df['Ticker'].unique()}

In [35]:
df.loc[df['Ticker'] == ticker, 'CDI'].values

array([2.76, 2.43])

In [40]:
for ticker in df['Ticker'].unique():
    if df.loc[df['Ticker'] == ticker, 'CDI'].isna().any():
        regr[ticker].fit(df.loc[df['Ticker'] == ticker, 'Valor'].values.reshape(-1, 1), 
                 df.loc[df['Ticker'] == ticker, 'IPCA'].values)
    else:
        regr[ticker].fit(df.loc[df['Ticker'] == ticker, 'Valor'].values.reshape(-1, 1), 
                 df.loc[df['Ticker'] == ticker, 'CDI'].values)

In [70]:
for ticker in df['Ticker'].unique():
    response = requests.get(f'https://brapi.dev/api/quote/{ticker}?range=1d&interval=1d&fundamental=false&dividends=false')
    result = json.loads(response.text)
    price = result['results'][0]['regularMarketPrice']
    previousClose = result['results'][0]['regularMarketPreviousClose']
    
    index = 'CDI '
    if df.loc[df['Ticker'] == ticker, 'CDI'].isna().any(): index = 'IPCA'
    
    print(f'{ticker} Preço Atual: {price:6.2f} Change: {(price / previousClose - 1) * 100:+4.2f} Carteira: {index} + {regr[ticker].predict(np.array(price).reshape(-1, 1))[0] :5.2f}')

KNIP11 Preço Atual:  92.86 Change: +0.47 Carteira: IPCA +  8.14
KNSC11 Preço Atual:  87.75 Change: -0.15 Carteira: IPCA +  7.71
KNHY11 Preço Atual:  99.97 Change: +1.18 Carteira: IPCA +  9.85
KCRE11 Preço Atual:  96.01 Change: -0.15 Carteira: IPCA +  8.28
KNCR11 Preço Atual:  98.10 Change: +0.25 Carteira: CDI  +  2.28
KNCA11 Preço Atual: 106.61 Change: -0.29 Carteira: CDI  +  2.23
